In [ ]:
!pip install pycuda

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 27.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.8/91.8 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 6.8 MB/s eta 0:00:00
  Created wheel for pycuda: filename=pycuda-2024.1.2-cp310-cp310-linux_x86_64.whl size=660545 sha256=d058d3e45ca2f5266faf4244242486ae7b0a155a8d6583f29aacff84e722866c
  Stored in directory: /root/.cache/pip/wheels/70/63/40/4bf006182f942d3516b71bb2ff3b57ccbdb8b2c0ee81882b6e
Successfully built pycuda


In [ ]:
"""Components for an actuator."""

import autograd.numpy as np


class Link(object):
    """Represents a link."""

    def __init__(self, coord):
        """Create a link from a specified coordinate."""
        self.coord = coord

    def matrix(self, _):
        """Return translation matrix in homogeneous coordinates."""
        x, y, z = self.coord
        return np.array([
            [1., 0., 0., x],
            [0., 1., 0., y],
            [0., 0., 1., z],
            [0., 0., 0., 1.]
        ])


class Joint(object):
    """Represents a revolute joint."""

    def __init__(self, axis):
        """Create a revolute joint from a specified axis."""
        self.axis = axis

    def matrix(self, angle):
        """Return rotation matrix in homogeneous coordinates."""
        _rot_mat = {
            'x': self._x_rot,
            'y': self._y_rot,
            'z': self._z_rot
        }
        return _rot_mat[self.axis](angle)

    def _x_rot(self, angle):
        return np.array([
            [1., 0., 0., 0.],
            [0., np.cos(angle), -np.sin(angle), 0.],
            [0., np.sin(angle), np.cos(angle), 0.],
            [0., 0., 0., 1.]
        ])

    def _y_rot(self, angle):
        return np.array([
            [np.cos(angle), 0., np.sin(angle), 0.],
            [0., 1., 0., 0.],
            [-np.sin(angle), 0., np.cos(angle), 0.],
            [0., 0., 0., 1.]
        ])

    def _z_rot(self, angle):
        return np.array([
            [np.cos(angle), -np.sin(angle), 0., 0.],
            [np.sin(angle), np.cos(angle), 0., 0.],
            [0., 0., 1., 0.],
            [0., 0., 0., 1.]
        ])


In [ ]:
# solver_cuda.py
import numpy as np
import pycuda.autoinit
import pycuda.driver as cuda
from pycuda.compiler import SourceModule

class CUDAFKSolver:
    def __init__(self, components):
        self.components = components
        self.joint_indexes = [
            i for i, c in enumerate(components) if isinstance(c, Joint)
        ]

        self.mod = SourceModule(r"""
        __global__ void fk_kernel(float *matrices, float *result, int num_components) {
            // Matrix chain multiplication
            float current[16] = {1,0,0,0, 0,1,0,0, 0,0,1,0, 0,0,0,1};
            float temp[16];

            // for (int i = num_components - 1; i >= 0; i--) {
            for (int i = 0; i < num_components; i++) {
                // Matrix multiplication
                for (int row = 0; row < 4; row++) {
                    for (int col = 0; col < 4; col++) {
                        float sum = 0.0f;
                        for (int k = 0; k < 4; k++) {
                            sum += current[row * 4 + k] * matrices[i * 16 + k * 4 + col];
                        }
                        temp[row * 4 + col] = sum;
                    }
                }
                // Copy result back to current
                for (int j = 0; j < 16; j++) {
                    current[j] = temp[j];
                }
            }

            // Store final position
            result[0] = current[3];
            result[1] = current[7];
            result[2] = current[11];
        }
        """)

        self.fk_kernel = self.mod.get_function("fk_kernel")

    def _matrices(self, angles):
        joints = dict(zip(self.joint_indexes, angles))
        a = [joints.get(i, None) for i in range(len(self.components))]
        return [c.matrix(a[i]) for i, c in enumerate(self.components)]

    def solve(self, angles):
        # Get transformation matrices (no need to reverse anymore)
        matrices = self._matrices(angles)
        matrices_flat = np.array([m.flatten() for m in matrices], dtype=np.float32)
        # print(matrices_flat)
        # Allocate GPU memory
        d_matrices = cuda.mem_alloc(matrices_flat.nbytes)
        d_result = cuda.mem_alloc(3 * np.float32().nbytes)

        # Copy matrices to GPU
        cuda.memcpy_htod(d_matrices, matrices_flat)
        # Launch kernel
        self.fk_kernel(d_matrices, d_result, np.int32(len(matrices)),
                      block=(32, 1, 1), grid=(1, 1))

        # Get result
        result = np.empty(3, dtype=np.float32)
        cuda.memcpy_dtoh(result, d_result)

        return result



class CUDAIKSolver:
    def __init__(self, fk_solver, optimizer):
        """Generate an IK solver from a FK solver instance."""
        def distance_squared(angles, target):
            x = target - fk_solver.solve(angles)
            print("Result: " + str(fk_solver.solve(angles)))
            return np.sum(np.power(x, 2))
        optimizer.prepare(distance_squared)
        self.optimizer = optimizer

    def solve(self, angles0, target):
        """Calculate joint angles and returns it."""
        return self.optimizer.optimize(np.array(angles0), target)




In [ ]:
"""Optimizers."""

import autograd.numpy as np
import autograd
import scipy.optimize


class NewtonOptimizer(object):
    """An optimizer based on Newton's method."""

    def __init__(self, tol=1.48e-08, maxiter=50):
        """Generate an optimizer from an objective function."""
        self.tol = tol
        self.maxiter = maxiter

    def prepare(self, f):
        """Accept an objective function for optimization."""
        self.g = autograd.grad(f)
        self.h = autograd.hessian(f)

    def optimize(self, x0, target):
        """Calculate an optimum argument of an objective function."""
        x = x0
        for _ in range(self.maxiter):
            delta = np.linalg.solve(self.h(x, target), -self.g(x, target))
            x = x + delta
            if np.linalg.norm(delta) < self.tol:
                break
        return x


class SteepestDescentOptimizer(object):
    """An optimizer based on steepest descent method."""

    def __init__(self, tol=1.48e-08, maxiter=50, alpha=1):
        """Generate an optimizer from an objective function."""
        self.tol = tol
        self.maxiter = maxiter
        self.alpha = alpha

    def prepare(self, f):
        """Accept an objective function for optimization."""
        self.g = autograd.grad(f)

    def optimize(self, x0, target):
        """Calculate an optimum argument of an objective function."""
        x = x0
        for _ in range(self.maxiter):
            delta = self.alpha * self.g(x, target)
            x = x - delta
            if np.linalg.norm(delta) < self.tol:
                break
        return x


class ConjugateGradientOptimizer(object):
    """An optimizer based on conjugate gradient method."""

    def __init__(self, tol=1.48e-08, maxiter=50):
        """Generate an optimizer from an objective function."""
        self.tol = tol
        self.maxiter = maxiter

    def prepare(self, f):
        """Accept an objective function for optimization."""
        self.g = autograd.grad(f)
        self.h = autograd.hessian(f)

    def optimize(self, x0, target):
        """Calculate an optimum argument of an objective function."""
        x = x0
        for i in range(self.maxiter):
            g = self.g(x, target)
            h = self.h(x, target)
            if i == 0:
                alpha = 0
                m = g
            else:
                alpha = - np.dot(m, np.dot(h, g)) / np.dot(m, np.dot(h, m))
                m = g + np.dot(alpha, m)
            t = - np.dot(m, g) / np.dot(m, np.dot(h, m))
            delta = np.dot(t, m)
            x = x + delta
            if np.linalg.norm(delta) < self.tol:
                break
        return x


class ScipyOptimizer(object):
    """An optimizer based on scipy.optimize.minimize."""

    def __init__(self, **optimizer_opt):
        """Generate an optimizer from an objective function."""
        for k, v in [  # default values
                ('method', 'BFGS'),
                ('tol', 1.48e-08),
                ('options', {'maxiter': 50})]:
            if k not in optimizer_opt:
                optimizer_opt[k] = v
        self.optimizer_opt = optimizer_opt

    def prepare(self, f):
        """Accept an objective function for optimization."""
        self.f = f

    def optimize(self, angles0, target):
        """Calculate an optimum argument of an objective function."""
        def new_objective(angles):
            return self.f(angles, target)

        return scipy.optimize.minimize(
            new_objective,
            angles0,
            **self.optimizer_opt).x


class ScipySmoothOptimizer(ScipyOptimizer):
    """A smooth optimizer based on scipy.optimize.minimize."""

    def __init__(self, smooth_factor=.1, **optimizer_opt):
        """Generate an optimizer from an objective function."""
        self.smooth_factor = smooth_factor
        for k, v in [  # default values
                ('method', 'L-BFGS-B'),
                ('bounds', None)]:
            if k not in optimizer_opt:
                optimizer_opt[k] = v
        super(ScipySmoothOptimizer, self).__init__(**optimizer_opt)

    def optimize(self, angles0, target):
        """Calculate an optimum argument of an objective function."""
        def new_objective(angles):
            a = angles - angles0
            if isinstance(self.smooth_factor, (np.ndarray, list)):
                if len(a) == len(self.smooth_factor):
                    return (self.f(angles, target) +
                            np.sum(self.smooth_factor * np.power(a, 2)))
                else:
                    raise ValueError('len(smooth_factor) != number of joints')
            else:
                return (self.f(angles, target) +
                        self.smooth_factor * np.sum(np.power(a, 2)))

        return scipy.optimize.minimize(
            new_objective,
            angles0,
            **self.optimizer_opt).x


In [ ]:
"""Core features."""

from numbers import Number

import autograd.numpy as np


class Actuator(object):
    """Represents an actuator as a set of links and revolute joints."""

    def __init__(self, tokens, optimizer=None):
        """Create an actuator from specified link lengths and joint axes."""
        components = []
        for t in tokens:
            if isinstance(t, Number):
                components.append(Link([t, 0., 0.]))
            elif isinstance(t, list) or isinstance(t, np.ndarray):
                components.append(Link(t))
            elif isinstance(t, str) and t in {'x', 'y', 'z'}:
                components.append(Joint(t))
            else:
                raise ValueError(
                    'the arguments need to be '
                    'link length or joint axis: {}'.format(t)
                )

        self.fk = CUDAFKSolver(components)
        self.ik = CUDAIKSolver(
            self.fk, ScipyOptimizer() if optimizer is None else optimizer)

        self.angles = [0.] * len(
            [c for c in components if isinstance(c, Joint)]
        )
        self.components = components

    @property
    def angles(self):
        """The joint angles."""
        return self._angles

    @angles.setter
    def angles(self, angles):
        self._angles = np.array(angles)

    @property
    def ee(self):
        """The end-effector position."""
        return self.fk.solve(self.angles)

    @ee.setter
    def ee(self, position):
        self.angles = self.ik.solve(self.angles, position)


In [ ]:
arm = Actuator(['z', [1., 0., 0.], 'z', [1., 0., 0.]])
print(arm.angles) # array([ 0.,  0.])
print(arm.ee) # array([ 2.,  0.,  0.])

import numpy as np
# arm.angles = [np.pi / 6, np.pi / 3]  # or np.deg2rad([30, 60])
# print(arm.ee) # array([ 0.8660254,  1.5      ,  0.       ])

arm.ee = [2 / np.sqrt(2), 2 / np.sqrt(2), 0.]
print(arm.angles)

[0. 0.]
[2. 0. 0.]
Result: [2. 0. 0.]
Result: [2.0000000e+00 2.9802322e-08 0.0000000e+00]
Result: [2.0000000e+00 1.4901161e-08 0.0000000e+00]
Result: [0.833035  1.7622366 0.       ]
Result: [0.8330349 1.7622366 0.       ]
Result: [0.83303493 1.7622366  0.        ]
Result: [1.8136923  0.82547146 0.        ]
Result: [1.8136923 0.8254715 0.       ]
Result: [1.8136923  0.82547146 0.        ]
Result: [0.8933761 1.7888432 0.       ]
Result: [0.8933761 1.7888432 0.       ]
Result: [0.8933761 1.7888432 0.       ]
[ 1.12952156 -0.0438104 ]


In [ ]:
import numpy as np

try:
    import open3d as o3d  # the extra feature
except ImportError:
    pass


def translate(p):
    x, y, z = p
    return np.array([
            [1., 0., 0., x],
            [0., 1., 0., y],
            [0., 0., 1., z],
            [0., 0., 0., 1.]
        ])


def rotate(axis, angle):
    x, y, z = axis
    return np.array([  # Rodrigues
            [
                np.cos(angle) + (x**2 * (1 - np.cos(angle))),
                (x * y * (1 - np.cos(angle))) - (z * np.sin(angle)),
                (x * z * (1 - np.cos(angle))) + (y * np.sin(angle)),
                0.
            ], [
                (y * x * (1 - np.cos(angle))) + (z * np.sin(angle)),
                np.cos(angle) + (y**2 * (1 - np.cos(angle))),
                (y * z * (1 - np.cos(angle))) - (x * np.sin(angle)),
                0.
            ], [
                (z * x * (1 - np.cos(angle))) - (y * np.sin(angle)),
                (z * y * (1 - np.cos(angle))) + (x * np.sin(angle)),
                np.cos(angle) + (z**2 * (1 - np.cos(angle))),
                0.
            ], [0., 0., 0., 1.]
        ])


def create_sphere(p, radius, color=None):
    if color is None:
        color = [.8, .8, .8]
    geo = o3d.geometry.TriangleMesh.create_sphere(radius=radius)
    geo.compute_vertex_normals()
    geo.paint_uniform_color(color)
    geo.transform(translate(p))
    return geo


class GeoComponent:

    child = None
    radius = .1

    def tip(self, link_color=None):
        return create_sphere(
            [0., 0., 0.],
            radius=self.radius*2,
            color=[.8, .8, .8] if link_color is None else link_color)

    def geo(self, mat=None, link_color=None):
        geo = self.base_geo(link_color)
        if mat is not None:
            geo.transform(mat)
            mat = mat @ self.mat()
        else:
            mat = self.mat()
        if self.child is None:
            return [geo] + [self.tip(link_color).transform(mat)]
        else:
            return [geo] + self.child.geo(mat, link_color)


class Link(GeoComponent):

    def __init__(self, c, radius):
        self.c = c
        self.radius = radius

    def base_geo(self, link_color=None):
        norm = np.linalg.norm(self.c.coord)

        geo = o3d.geometry.TriangleMesh.create_cylinder(
            radius=self.radius, height=norm)
        geo.compute_vertex_normals()
        geo.paint_uniform_color(
            [.8, .8, .8] if link_color is None else link_color)

        # Calculate transformation matrix for cylinder
        # With help from https://stackoverflow.com/a/59829173
        def get_cross_prod_mat(vector):
            return np.array([
                [0, -vector[2], vector[1]],
                [vector[2], 0, -vector[0]],
                [-vector[1], vector[0], 0],
            ])
        cylinder_dir_unit_vector = self.c.coord / norm

        # Unit vector for "up" direction
        z_unit_vector = np.array([0, 0, 1])
        z_rotation_mat = get_cross_prod_mat(z_unit_vector)

        z_c_vec = np.matmul(z_rotation_mat, cylinder_dir_unit_vector)
        z_c_vec_mat = get_cross_prod_mat(z_c_vec)

        # Added np.abs to ensure that unit vector that is aligned with any
        # axis in the negative direction does not
        rotation_mat = np.eye(3, 3) + z_c_vec_mat + np.matmul(
            z_c_vec_mat, z_c_vec_mat)/(
                (1 + np.abs(np.dot(z_unit_vector, cylinder_dir_unit_vector))))

        cylinder_transform_mat = np.vstack((np.hstack(
            (rotation_mat, np.transpose(
                np.array([self.c.coord])/2))), np.array([0, 0, 0, 1])))

        geo.transform(cylinder_transform_mat)
        return geo

    def mat(self):
        return self.c.matrix(None)


class Joint(GeoComponent):

    def __init__(self, c, radius):
        self.c = c
        self.radius = radius
        self.angle = 0.

    def base_geo(self, _=None):
        geo = o3d.geometry.TriangleMesh.create_cylinder(
            radius=self.radius*2, height=self.radius*4)
        geo.compute_vertex_normals()
        geo.paint_uniform_color([.2, .2, .9])
        rx = {
            'x': [0., 1., 0.],
            'y': [1., 0., 0.],
            'z': [0., 0., 1.],
        }
        geo.transform(rotate(rx[self.c.axis], np.pi / 2))
        return geo

    def mat(self):
        return self.c.matrix(self.angle)


def build_geos(actuator, target=None, radius=.05):
    root = None
    p = None
    joints = []
    for c in actuator.components:
        if hasattr(c, 'axis'):
            gc = Joint(c, radius)
            joints.append(gc)
        else:
            gc = Link(c, radius)

        if root is None:
            root = gc
            p = gc
        else:
            p.child = gc
            p = gc

    for j, a in zip(joints, actuator.angles):
        j.angle = a

    if target:
        geos = root.geo(link_color=[.5, .5, .5])
        actuator.ee = target
        for j, a in zip(joints, actuator.angles):
            j.angle = a
        geos += root.geo()
        geos += [create_sphere(target, radius=radius*2.4, color=[.8, .2, .2])]
    else:
        geos = root.geo()

    return geos


def visualize(actuator, target=None, radius=.05):
    geos = build_geos(actuator, target, radius)
    o3d.visualization.draw_geometries(
        geos, window_name='tinyik vizualizer', width=640, height=480)


In [ ]:
leg = Actuator([[.3, .0, .0], 'z', [.3, .0, .0], 'x', [.0, -.5, .0], 'x', [.0, -.5, .0]])
leg.angles = np.deg2rad([30, 45, -90])
visualize(leg)

TypeError: Link.__init__() missing 1 required positional argument: 'radius'

In [ ]:
self.visualize(leg, target=[.8, .0, .8])